In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import numpy as np
import bidi.algorithm
import arabic_reshaper
import adhtools.utils

In [2]:
fpath = '/media/sf_VBox_Shared/Arabic/Analyses/Fiqh_final2/topicmodeling/fiqh-light10-20/'

In [3]:
document_topics = pd.read_csv(os.path.join(fpath, 'document_topics_20.csv'), index_col=0).sort_index()
df_topics = pd.read_csv(os.path.join(fpath, 'topics_20.csv'), index_col=0)

In [5]:
df_topics_old = pd.read_csv('/media/sf_VBox_Shared/Arabic/Analyses/Fiqh_final/topicmodelling/fiqh-light10-20/fiqh_light10_topics_20_labeled.csv', index_col=0)

In [52]:
df_topics_old

,0,1,2,3,4,5,6,7,8,9,topic_level1,topic_level2
topic_nr,,,,,,,,,,,,
0,عقد,فل,عد,ولد,اذ,مهر,ام,نكاح,ول,زوج,personal law,marriage
1,اخذ,اذ,وقف,خمس,عشر,حول,صدق,ول,مال,زكا,ritual,aims-giving
2,سجد,سلام,ركع,وقت,اول,صل,ول,اذ,امام,صلا,ritual,prayer
3,امام,فل,اهل,قطع,اسلام,اذ,قتل,حد,مسلم,ول,criminal law,punishments
4,خيار,مال,بااع,عقد,ول,قبض,اذ,ثمن,مشتر,بيع,contracts,sales
5,ول,ظاهر,عدم,هذ,بخلاف,ام,اذ,فل,ذكر,اول,noise,NaN
6,طهار,مسح,حيض,دم,نجاس,اذ,صلا,وض,ول,غسل,ritual,purity
7,شافع,ابو,رجل,نب,حديث,مالك,عمر,رسول,اب,صل,transmission,prophetic hadith and early legal authorities
8,ذكر,صحيح,اب,ظاهر,اول,اخبار,شيخ,روا,هذ,سلام,transmission,general


In [17]:
df_topics_old_sub = df_topics_old[df_topics_old.columns[0:10]]

In [19]:
set(df_topics_old_sub.loc[15])

{'اذ', 'ثلاث', 'حد', 'حلف', 'حنث', 'طالق', 'طلاق', 'طلق', 'ول', 'يم'}

In [35]:
## compute overlap
import numpy as np
nr_topics = 20
overlaps_df = pd.DataFrame()
for i in range(nr_topics):
    for j in range(nr_topics):
        s_old = set(df_topics_old_sub.loc[i])
        s_new = set(df_topics.loc[j])
        overlaps_df.loc[i,j] = len(s_old.intersection(s_new))
        
overlaps_df.index.name = 'old'
overlaps_df.columns.name = 'new'

In [39]:
overlaps_df.style.background_gradient()

new,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
old,,,,,,,,,,,,,,,,,,,,
0,2,2,3,2,3,2,8,2,2,2,2,3,1,0,2,3,0,1,2,3
1,2,2,2,2,2,2,2,2,2,1,8,2,1,0,1,4,0,2,2,2
2,3,2,2,3,2,9,2,2,2,1,2,3,1,1,2,2,1,0,2,1
3,7,5,3,2,2,3,4,2,3,2,3,3,1,0,1,3,0,0,2,1
4,2,2,2,2,9,2,2,2,2,1,3,2,1,0,1,3,0,1,2,2
5,2,2,4,2,3,4,4,2,3,5,2,7,2,3,4,3,0,1,2,2
6,2,2,2,9,2,3,2,3,2,1,2,2,1,0,1,2,0,0,2,1
7,0,1,1,0,0,1,0,1,0,1,0,0,2,1,0,0,10,1,0,0
8,0,0,1,0,1,3,0,0,1,3,0,4,1,8,2,0,1,1,0,0


In [40]:
# OVerlap bigger than 7?
def color_value(val, threshold):
    if val >= threshold:
        return 'background-color: green' 
    else:
        return ''

overlaps_df.style.applymap(lambda s: color_value(s,7))

new,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
old,,,,,,,,,,,,,,,,,,,,
0,2,2,3,2,3,2,8,2,2,2,2,3,1,0,2,3,0,1,2,3
1,2,2,2,2,2,2,2,2,2,1,8,2,1,0,1,4,0,2,2,2
2,3,2,2,3,2,9,2,2,2,1,2,3,1,1,2,2,1,0,2,1
3,7,5,3,2,2,3,4,2,3,2,3,3,1,0,1,3,0,0,2,1
4,2,2,2,2,9,2,2,2,2,1,3,2,1,0,1,3,0,1,2,2
5,2,2,4,2,3,4,4,2,3,5,2,7,2,3,4,3,0,1,2,2
6,2,2,2,9,2,3,2,3,2,1,2,2,1,0,1,2,0,0,2,1
7,0,1,1,0,0,1,0,1,0,1,0,0,2,1,0,0,10,1,0,0
8,0,0,1,0,1,3,0,0,1,3,0,4,1,8,2,0,1,1,0,0


In [50]:
overlaps_s = overlaps_df.unstack()
overlaps_s = overlaps_s[overlaps_s>=7]
overlaps_s

new  old
0    3       7.0
1    10      8.0
2    13     10.0
3    6       9.0
4    4       9.0
5    2       9.0
6    0       8.0
7    19      8.0
8    16      9.0
9    9       7.0
10   1       8.0
11   5       7.0
12   12      7.0
13   8       8.0
15   11      7.0
16   7      10.0
17   14      7.0
dtype: float64

In [58]:
for i, j in list(overlaps_s.index):
    print(i, '<--', j)
    lev1, lev2 = tuple(df_topics_old.loc[j][['topic_level1', 'topic_level2']])
    print('{}\t{}'.format(lev1, lev2))

0 <-- 3
criminal law	punishments
1 <-- 10
criminal law	homicide
2 <-- 13
transmission	Mālikī authorities
3 <-- 6
ritual	purity
4 <-- 4
contracts	sales
5 <-- 2
ritual	prayer
6 <-- 0
personal law	marriage
7 <-- 19
ritual	pilgrimage and fasting
8 <-- 16
procedure	nan
9 <-- 9
transmission	Ḥanafī authorities
10 <-- 1
ritual	aims-giving
11 <-- 5
noise	nan
12 <-- 12
noise	nan
13 <-- 8
transmission	general
15 <-- 11
contracts	estates
16 <-- 7
transmission	prophetic hadith and early legal authorities
17 <-- 14
personal law	inheritance


In [59]:
document_topics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0179MalikIbnAnas.Muwatta-00000.xml,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.220000,0.020000,0.020000,0.020000,0.020000,0.220000,0.020000,0.020000,0.220000,0.020000,0.020000
0179MalikIbnAnas.Muwatta-00001.xml,0.000258,0.000258,0.013178,0.604910,0.002842,0.000258,0.000258,0.000258,0.000258,0.000258,0.000258,0.000258,0.000258,0.000258,0.374935,0.000258,0.000258,0.000258,0.000258,0.000258
0179MalikIbnAnas.Muwatta-00002.xml,0.001818,0.001818,0.001818,0.801818,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.147273,0.001818,0.001818,0.020000,0.001818,0.001818
0179MalikIbnAnas.Muwatta-00003.xml,0.001724,0.001724,0.001724,0.536207,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.001724,0.432759,0.001724,0.001724,0.001724,0.001724,0.001724
0179MalikIbnAnas.Muwatta-00004.xml,0.000680,0.000680,0.000680,0.517687,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.470068,0.000680,0.000680,0.000680,0.000680,0.000680


In [63]:
def get_page_for_doc(fname):
    base_url = 'http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs'
    url = base_url + '?filter=fromInputFile:(*{})'.format(fname)
    return url

In [64]:
for topic_nr in document_topics.columns:
    print('Topic {}'.format(topic_nr))
    print(' '.join(df_topics.loc[int(topic_nr),:]))
    important_docs = document_topics[topic_nr].sort_values(ascending=False).head(10).index
    for doc in important_docs:
        print(get_page_for_doc(doc))
    print('-------\n')

Topic 0
اذ دار علي قتل امام حرب اهل اسلام ول مسلم
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00200.xml)
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00117.xml)
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00190.xml)
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00192.xml)
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00122.xml)
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00143.xml)
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00124.xml)
http://arabic-dh.hum.uu.nl/corpus-frontend/Fiqh/search/docs?filter=fromInputFile:(*0189AlShaybani.alAsl-00181.xml)
http://arabic-dh.hum.uu.nl/cor